# Эксперименты с разными наборами признаков

## Создание временных признаков

На основе временных признаков можно создать новые полезные и хорошо интерпретируемые:
1. Время между оплатой заказа и подтверждением заказа на сайте (order_approved_at - order_purchase_timestamp): *payment_lag*
2. Время между подтверждением заказа и ориентировочным времени доставки (order_estimated_delivery_date - order_aproved_at): *estimated_shipping_lag*
3. Время между фактическим времени доставки и ожидаемым (order_delivered_customer_date - order_estimated_delivery_date): *actual_and_estimated_shipping_lag*
4. Время между рассылкой с просьбой оценить качество и ответом покупателя (review_answer_timestamp - review_creation_date): *customer_review_response_lag*
5. Время между подтверждением заказа и его прибытием (order_delivered_customer_date - order_aproved_at): *approval_and_arrival_lag*

In [ ]:
diffs = FeatureUnion([
    ('payment_lag', DateDiffTransformer('order_purchase_timestamp',
                                        'order_aproved_at',
                                        'payment_lag_hours')),
    ('estimated_shipping_lag', DateDiffTransformer('order_aproved_at',
                                         'order_estimated_delivery_date',
                                         'estimated_shipping_lag_hours')),
    ('actual_and_estimated_shipping_lag', DateDiffTransformer('order_estimated_delivery_date',
                                         'order_delivered_customer_date',
                                         'actual_and_estimated_shipping_lag_hours')),
    ('customer_review_response_lag', DateDiffTransformer('review_creation_date',
                                         'review_answer_timestamp',
                                         'customer_review_response_lag_hours')),
    ('approval_and_arrival_lag', DateDiffTransformer('order_aproved_at',
                                         'order_delivered_customer_date',
                                         'approval_and_arrival_lag_hours')),
])

In [ ]:
diff_matrix = diffs.fit_transform(df)

In [ ]:
print(diff_matrix)

In [ ]:
new_cols = [tr.new_col for _, tr in diffs.transformer_list]
classified_data_diffs  = pd.DataFrame(diff_matrix,
                         columns=new_cols,
                         index=df.index)

In [ ]:
df = df.join(classified_data_diffs)

In [ ]:
display(df.head())

In [ ]:
classified_data_time = df[['most_voted_class', 'most_voted_subclass', 'payment_lag_hours', 'estimated_shipping_lag_hours', 'actual_and_estimated_shipping_lag_hours', 'customer_review_response_lag_hours', 'approval_and_arrival_lag_hours']]